In [1]:
import pandas as pd
import numpy as np
import os
import glob
import shutil
import xml.etree.ElementTree as ET
import random

In [2]:
path ='../data/annotations.xml'
dataset = []

for anno in glob.glob(path):
    tree = ET.parse(anno)
    root = tree.getroot()
    
    for image_elem in root.iter("image"):
        image_attrs = image_elem.attrib
        image_data = {
            "filename": image_attrs['name'],
            "width": float(image_attrs['width']),
            "height": float(image_attrs['height']),
            "boxes": []
        }
        
        for box_elem in image_elem.iter("box"):
            box_attrs = box_elem.attrib
            box_data = {
                "label": box_attrs['label'],
                "occluded": int(box_attrs['occluded']),
                "xtl": float(box_attrs['xtl']),
                "ytl": float(box_attrs['ytl']),
                "xbr": float(box_attrs['xbr']),
                "ybr": float(box_attrs['ybr'])
            }
            image_data["boxes"].append(box_data)
        
        dataset.append(image_data)

In [3]:
df = pd.DataFrame(dataset)
df.head()

,filename,width,height,boxes
0,images/0.png,1200.0,800.0,"[{'label': 'strawberry', 'occluded': 0, 'xtl':..."
1,images/1.png,1600.0,900.0,"[{'label': 'strawberry', 'occluded': 1, 'xtl':..."
2,images/10.png,920.0,425.0,"[{'label': 'strawberry', 'occluded': 1, 'xtl':..."
3,images/11.png,640.0,480.0,"[{'label': 'strawberry', 'occluded': 0, 'xtl':..."
4,images/12.png,1920.0,1080.0,"[{'label': 'strawberry', 'occluded': 0, 'xtl':..."


In [4]:
df_=pd.DataFrame()
for i in range(len(df)):
    boxes=df.loc[i,'boxes']
    for box in boxes:
        a=df.loc[i,'filename']
        b=df.loc[i,'width']
        c=df.loc[i,'height']
        d=box['xtl']
        e=box['ytl']
        f=box['xbr']
        g=box['ybr']
        add_df=pd.DataFrame([[a,b,c,d,e,f,g]])
        df_=pd.concat([df_,add_df],axis=0)

df_.columns=['filename','width','height','xtl','ytl','xbr','ybr']
df_

,filename,width,height,xtl,ytl,xbr,ybr
0,images/0.png,1200.0,800.0,368.18,292.79,460.65,454.46
0,images/0.png,1200.0,800.0,423.70,268.56,537.50,471.16
0,images/0.png,1200.0,800.0,538.93,338.40,666.51,509.38
0,images/0.png,1200.0,800.0,642.29,416.47,789.86,616.35
0,images/0.png,1200.0,800.0,734.64,416.40,837.78,597.06
...,...,...,...,...,...,...,...
0,images/9.png,1600.0,1200.0,1208.45,713.03,1386.04,922.00
0,images/9.png,1600.0,1200.0,126.12,207.98,277.09,431.25
0,images/9.png,1600.0,1200.0,136.68,555.56,416.67,900.57
0,images/9.png,1600.0,1200.0,261.24,523.59,490.52,805.78


In [5]:
df_['label']=0
df_['Xcent']=(df_['xtl']+df_['xbr'])/(2*df_['width'])
df_['Ycent']=(df_['ytl']+df_['ybr'])/(2*df_['height'])
df_['boxW']=(df_['xbr']-df_['xtl'])/df_['width']
df_['boxH']=(df_['ybr']-df_['ytl'])/df_['height']
df_

,filename,width,height,xtl,ytl,xbr,ybr,label,Xcent,Ycent,boxW,boxH
0,images/0.png,1200.0,800.0,368.18,292.79,460.65,454.46,0,0.345346,0.467031,0.077058,0.202087
0,images/0.png,1200.0,800.0,423.70,268.56,537.50,471.16,0,0.400500,0.462325,0.094833,0.253250
0,images/0.png,1200.0,800.0,538.93,338.40,666.51,509.38,0,0.502267,0.529863,0.106317,0.213725
0,images/0.png,1200.0,800.0,642.29,416.47,789.86,616.35,0,0.596729,0.645513,0.122975,0.249850
0,images/0.png,1200.0,800.0,734.64,416.40,837.78,597.06,0,0.655175,0.633412,0.085950,0.225825
...,...,...,...,...,...,...,...,...,...,...,...,...
0,images/9.png,1600.0,1200.0,1208.45,713.03,1386.04,922.00,0,0.810778,0.681262,0.110994,0.174142
0,images/9.png,1600.0,1200.0,126.12,207.98,277.09,431.25,0,0.126003,0.266346,0.094356,0.186058
0,images/9.png,1600.0,1200.0,136.68,555.56,416.67,900.57,0,0.172922,0.606721,0.174994,0.287508
0,images/9.png,1600.0,1200.0,261.24,523.59,490.52,805.78,0,0.234925,0.553904,0.143300,0.235158


In [6]:
df_.to_csv('../dataset/annotation.csv',index=False)

In [7]:
files=df_['filename'].unique().tolist()
for file in files:
    datai=df_[df_['filename']==file].iloc[:,7:]
    name=file.split('/')[-1][0:-4]
    fmt = ['%d'] + ['%f'] * (datai.shape[1]-1)
    np.savetxt("../data/labels/"+name+".txt", np.array(datai), fmt=fmt, delimiter="\t")

In [8]:
train_path = '../dataset/train'
train_img_path = '../dataset/train/images'
train_labels_path = '../dataset/train/labels'
test_path = '../dataset/test'
test_img_path = '../dataset/test/images'
test_labels_path = '../dataset/test/labels'
valid_path = '../dataset/valid'
valid_img_path = '../dataset/valid/images'
valid_labels_path = '../dataset/valid/labels'

In [9]:
for p in [train_path, train_img_path, train_labels_path, test_path, test_img_path, test_labels_path, valid_path, valid_img_path, valid_labels_path]:
    os.makedirs(p, exist_ok=True)

In [10]:
img_path = '../data/images'
paths=[]
for dirname, _, filenames in os.walk(img_path):
    for filename in filenames:
        paths+=[(os.path.join(dirname, filename))]
paths=random.sample(paths,len(paths))
print(paths[0:5])

['../data/images\\29.png', '../data/images\\7.png', '../data/images\\8.png', '../data/images\\33.png', '../data/images\\24.png']


In [11]:
for i, path in enumerate(paths):
    try:
        # Determine the destination directories based on the index
        if i < 25:
            dest_dir_img = train_img_path
            dest_dir_label = train_labels_path
        elif i < 35:
            dest_dir_img = valid_img_path
            dest_dir_label = valid_labels_path
        else:
            dest_dir_img = test_img_path
            dest_dir_label = test_labels_path

        shutil.copy(path, dest_dir_img)

        base_filename = os.path.basename(path)
        filename_without_ext = os.path.splitext(base_filename)[0]
        label_file = filename_without_ext + '.txt'
        label_source_path = os.path.join('../data/labels', label_file)

        if os.path.exists(label_source_path):
            shutil.copy(label_source_path, dest_dir_label)
        else:
            print(f"Warning: Label file not found and was not copied: {label_source_path}")

    except FileNotFoundError as e:
        print(f"Error: File not found - {e}. Skipping this iteration.")
    except Exception as e:
        print(f"An unexpected error occurred at index {i} for path {path}: {e}")

In [12]:
import torch
from ultralytics import YOLO
from PIL import Image

In [13]:
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:01<00:00, 4.54MB/s]


In [19]:
train_results = model.train(
    data='../dataset/dataset.yml',  # Path to dataset configuration file
    epochs=100,  # Number of training epochs
    imgsz=640,  # Image size for training
    device="0",  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
    workers=0,
    project='../model'
)

Ultralytics 8.3.160  Python-3.11.7 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../dataset/dataset.yml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pos

 40%|███▉      | 2.12M/5.35M [00:01<00:02, 1.60MB/s]


KeyboardInterrupt: 

In [15]:
metrics = model.val()

Ultralytics 8.3.160  Python-3.11.7 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)


YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 1290.8156.4 MB/s, size: 679.8 KB)


val: Scanning C:\Users\venna\Desktop\deep_learning_practise\computer_vision\object_detection_segmentation\Ripped-Strawberry-Dectection\dataset\valid\labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.93s/it]


                   all         25        283      0.924        0.9      0.961      0.771
Speed: 1.0ms preprocess, 44.4ms inference, 0.0ms loss, 5.4ms postprocess per image
Results saved to runs\detect\train2


In [16]:
results = model("../dataset/test/images/5.png")  # Predict on an image
results[0].show()  # Display results

image 1/1 c:\Users\venna\Desktop\deep_learning_practise\computer_vision\object_detection_segmentation\Ripped-Strawberry-Dectection\notebook\..\dataset\test\images\5.png: 640x640 9 ripped_strawberrys, 33.0ms
Speed: 5.9ms preprocess, 33.0ms inference, 15.8ms postprocess per image at shape (1, 3, 640, 640)


In [18]:
model.export(format="onnx", dynamic=True)

Ultralytics 8.3.160  Python-3.11.7 torch-2.3.1+cu118 CPU (AMD Ryzen 7 5800H with Radeon Graphics)

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.56', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
WARNING Retry 1/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0,<1.18.0" "onnxslim>=0.1.56" "onnxruntime-gpu" ' returned non-zero exit status 1.
WARNING Retry 2/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0,<1.18.0" "onnxslim>=0.1.56" "onnxruntime-gpu" ' returned non-zero exit status 1.
WARNING requirements:  Command 'pip install --no-cache-dir "onnx>=1.12.0,<1.18.0" "onnxslim>=0.1.56" "onnxruntime-gpu" ' returned non-zero exit status 1.

ONNX: starting export with onnx 1.18.0 opset 17...
WARNING ONNX: simplifier failure: No module named 'onnxslim'
ONNX: export success  5.2s, saved as 'run

'runs\\detect\\train\\weights\\best.onnx'